In [20]:
import os
import pandas as pd 
import polars as pl # like pandas, but much faster
import polars.selectors as cs
import numpy as np
import os, shutil, glob
from random import randint
import re, math
import datetime
import gc
from pathlib import Path

os.getcwd()


'/scratch2-shared/david/colopaint3D/python'

### Read data

In [21]:

sourceDir = '/share/data/cellprofiler/automation/results'
rootDir = '/home/jovyan/scratch2-shared/david/colopaint3D'


### Input directories

In [22]:
OutputDir = 'data/1_FeaturesImages'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
NameContains = ''
InputFolders = pl.read_csv('../settings/filemap.csv')
print(InputFolders )
            


shape: (3, 1)
┌───────────────────┐
│ feature_names     │
│ ---               │
│ str               │
╞═══════════════════╡
│ featICF_nuclei    │
│ featICF_cells     │
│ featICF_cytoplasm │
└───────────────────┘


In [23]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2024-08-09 15:05:25


In [24]:
def print_time(msg=None):
    now = datetime.datetime.now()
    print(now.strftime('%Y-%m-%d %H:%M:%S'),msg or "")
float_columns=[pl.col(pl.Float32),pl.col(pl.Float64)]

In [25]:
metaEx = pl.read_csv(f'{rootDir}/settings/spher-colo52-v1-import-inputFiles-and-PLAIDresults.csv')


In [26]:
metaEx.head()
barcodes = metaEx.select(pl.col('barcode')).unique()
barcodes = barcodes['barcode']
barcodes.to_list()
dirlist = [f'{sourceDir}/{barcode}' for barcode in barcodes]


In [27]:
# metadmso = metaEx.filter(pl.col('Compound_ID')=='DMSO')

In [28]:
metaEx.head()

index,layout_id,well_id,image_id,cp_id,barcode,plate_well,cmpd_code,cmpdname,solvent,cmpd_conc,cmpd_conc_unit,stock_conc,stock_conc_unit,cmpd_vol,cmpd_vol_unit,well_vol,well_vol_unit,pert_type,article_id,target,pathway,pubchemID,smiles,inkey,target_type,clinical_status,cell_line
i64,str,str,i64,i64,str,str,str,str,str,f64,str,f64,str,f64,str,f64,str,str,str,str,str,f64,str,str,str,str,str
0,"""spher-colo52-v…","""B02""",4185,5532,"""PB000137""","""PB000137_B02""","""colo-006""","""PD0325901""","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1036""","""MEK""","""MAPK""",9.826528e6,"""OCC(O)CONC(=O)…","""SUDAHWBOROXANE…","""Targeted""","""Phase 2""","""HCT116"""
1,"""spher-colo52-v…","""B03""",4185,5532,"""PB000137""","""PB000137_B03""","""colo-018""","""Paclitaxel""","""dmso""",0.1,"""µM""",0.1,"""mM""",100.0,"""nl""",40.0,"""µl""","""trt""","""S1150""","""Autophagy,Micr…","""Cytoskeletal S…",441276.0,"""CC(=O)OC1C(=O)…","""RCINICONZNJXQF…","""Cytotoxic""","""Preclinical""","""HCT116"""
2,"""spher-colo52-v…","""B04""",4185,5532,"""PB000137""","""PB000137_B04""","""colo-009""","""Olaparib (AZD2…","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1060""","""PARP""","""DNA Damage""",2.3725625e7,"""FC1=C(C=C(CC2=…","""FDLYAMZZIXQODN…","""Targeted""","""Launched""","""HCT116"""
3,"""spher-colo52-v…","""B05""",4185,5532,"""PB000137""","""PB000137_B05""","""colo-012""","""SB216763""","""dmso""",10.0,"""µM""",10.0,"""mM""",100.0,"""nl""",40.0,"""µl""","""trt""","""S1075""","""GSK-3""","""PI3K/Akt/mTOR""",176158.0,"""C[N]1C=C(C2=C1…","""JCSGFHVFHSKIJH…","""Targeted""","""Preclinical""","""HCT116"""
4,"""spher-colo52-v…","""B06""",4185,5532,"""PB000137""","""PB000137_B06""","""colo-008""","""Vorinostat (SA…","""dmso""",3.0,"""µM""",10.0,"""mM""",30.0,"""nl""",40.0,"""µl""","""trt""","""S1047""","""Autophagy,HDAC…","""Epigenetics""",5311.0,"""ONC(=O)CCCCCCC…","""WAEXFXRVDQXREF…","""Targeted""","""Launched""","""HCT116"""


In [29]:
cols_to_drop = ['index','layout_id','cmpd_code', 'solvent','cmpd_conc','cmpd_conc_unit','stock_conc','stock_conc_unit','cmpd_vol', 'cmpd_vol_unit', 'well_vol', 'well_vol_unit', 'article_id', 'target','pubchemID', 'smiles', 'inkey', 'clinical_status']
metaEx = metaEx.drop(cols_to_drop)

In [30]:
metaEx.select(pl.col(['cell_line'])).unique().to_series().to_list()

['HCT116', 'HT29']

In [31]:
use_clipping = False

std_mean = True

In [32]:
# df.head()

In [33]:
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']

for cl in metaEx.select(pl.col(['cell_line'])).unique().to_series().to_list():
    print('lala')
    df = pl.DataFrame()
    for BaseDir in dirlist:
        mdf_op = metaEx.filter(pl.col('barcode') == BaseDir.split('/')[-1])
        image_id = mdf_op.select(pl.col('image_id')).unique().to_series().to_list()[-1]
        cp_id = mdf_op.select(pl.col('cp_id')).unique().to_series().to_list()[-1]
        print(f'{BaseDir}/{image_id}/{cp_id}')
        BaseDir = f'{BaseDir}/{image_id}/{cp_id}'


        # nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
        # cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet').add_prefix('Cytoplasm_').reset_index()
        # cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()
        # f_df=pl.read_parquet(f)
        # f_df=f_df.rename({x:f'{feature_set_name}_{x}' for x in f_df.columns})
        

        nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet')
        nuclei=nuclei.rename({x:f'Nuclei_{x}' for x in nuclei.columns})
        cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet')
        cytoplasm=cytoplasm.rename({x:f'Cytoplasm_{x}' for x in cytoplasm.columns})
        cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet')
        cells=cells.rename({x:f'Cells_{x}' for x in cells.columns})
        # step 1: Take the mean values of 'multiple nuclei' belonging to one cell

        nuclei = nuclei.group_by([
            "Nuclei_Metadata_Barcode","Nuclei_Metadata_Well",
            "Nuclei_Parent_cells", 'Nuclei_Metadata_Site'
        ]).mean()

        df_one = cytoplasm.join(nuclei,
                    how='left', 
                    right_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Site', 'Nuclei_Parent_cells', 'Nuclei_Metadata_Barcode'],
                    left_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site', 'Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'])
                    
        df_one = df_one.join(cells, how='left', 
                        left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site','Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'],
                        right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])

        # print_time("initial merging")
        print('part1')

        # deduplicate barcode/well/site - renamed cytoplasm_Metadata* to Metadata* and removes nuclei_* etc
        unique_metadata_feature_names=['Metadata_Barcode','Metadata_Well','Metadata_Site']
        df_one=df_one.rename({f'Cytoplasm_{suffix}':suffix for suffix in unique_metadata_feature_names})
        # df = df.filter(pl.col(''))       
        # for some reason, the site is parsed as float, even though it really should be an int
        if df_one['Metadata_Site'].dtype in [np.dtype('float32'), np.dtype('float64')]:
            # sometimes, for some reason, site indices are inf/nan
            site_is_nan_mask=np.isnan(df_one['Metadata_Site'])
            site_is_inf_mask=np.isinf(df_one['Metadata_Site'])
            
            try:
                num_sites_nan=np.sum(site_is_nan_mask)
                num_sites_inf=np.sum(site_is_inf_mask)
                assert num_sites_nan==0, f"found nan site values (n = {num_sites_nan})"
                assert num_sites_inf==0, f"found inf site values (n = {num_sites_inf})"
            except AssertionError as e:
                print(f"info - this issue was automatically circumvented in the code : {e}")
                df_one=df_one[~(site_is_inf_mask|site_is_nan_mask)]
                
            num_metadata_site_entries_nonint=np.sum(np.abs(df_one['Metadata_Site']%1.0)>1e-6)
            assert num_metadata_site_entries_nonint==0, f"ERROR : {num_metadata_site_entries_nonint} imaging sites don't have integer indices. that should not be the case, and likely indicates a bug."
            
            #Should use np.round, no? TODO ask patrick. Truncation Errors are annoying.
            df_one['Metadata_Site'] = df_one['Metadata_Site'].astype(np.dtype('int32'))
        
        #Adding Compound Metadata to each row
        df_one = df_one.join(mdf_op.rename({x:f"Metadata_cmpd_{x}" for x in mdf_op.columns}),left_on='Metadata_Well',right_on='Metadata_cmpd_well_id')
        df_one = df_one.filter(pl.col('Metadata_Site')<7)
        df = pl.concat([df, df_one])
        plate_name = f'processed metadata for {BaseDir.split("/")[-1]}'
        print(plate_name)
    ###Here should be workable to unify by cell line.
    df = df.with_columns((pl.col("Metadata_Barcode") + "_" + pl.col("Metadata_Well")).alias("Metadata_PlateWell"))
    df = df.filter(pl.col('Metadata_cmpd_cell_line')==cl)
    ###End
    # drop all rows that contain nan
    num_rows_before_nan_trim = df.shape[0]
    for col in df.select([pl.col(pl.Float32),pl.col(pl.Float64)]).columns:
        before_drop=df.shape[0]
        df=df.filter(pl.col(col).is_not_null())
        after_drop=df.shape[0]

        #num_values_dropped=before_drop-after_drop
        #if num_values_dropped>0:
        #    print(f"dropped {num_values_dropped} rows due to NaNs in column {col}")

    num_rows_after_nan_trim = df.shape[0]

    print_time("dropped NaNs")

    # Clip outliers
    if use_clipping:
        float_cols = [c for c_name,c_dtype in zip(df.columns,df.dtypes) if "float" in str(c_dtype)]
        lower_quantile = df.select(float_cols).quantile(0.01)
        upper_quantile = df.select(float_cols).quantile(0.99)
        print("calced quantiles")

        for col in float_cols:
            df = df.with_column(
                pl.col(col).clip(lower=lower_quantile[col],upper=upper_quantile[col])
            )
            
        print("clipped")
    # print(df.head())
    # for some reason, DMSO is used as batch id for cells that are not treated with a drug
    df_DMSO=df.filter(pl.col('Metadata_cmpd_cmpdname')=='dmso')
    assert df_DMSO.shape[0]>0, "did not find any wells 'treated' with DMSO"

    # Running Means here:
    mu = df_DMSO.select(float_columns).mean()
    for col in mu.columns:
        if mu[col].is_null().any():
            raise RuntimeError(f"some mean value in column {col,i} is nan?!")
        if mu[col].is_infinite().any():
            raise RuntimeError(f"some mean value in column {col,i} is infinite?!")
                
    std = df_DMSO.select(float_columns).std()
    # replace 0 with 1 (specifically not clip) to avoid div by zero
    std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])

    for i,col in enumerate(std.columns):
        if std[col].is_null().any():
            raise RuntimeError(f"some std value in column {col,i} is nan?!")
        if std[col].is_infinite().any():
            raise RuntimeError(f"some std value in column {col,i} is infinite?!")
        if (std[col]==0).any():
            raise RuntimeError(f"unexpected 0 in column {col}")

    print_time("calculated DMSO distribution")

    df_standardized = df.with_columns([(pl.col(c) - mu[c]) / (std[c]+0.01) for c in mu.columns])
    found_nan=False
    # checking nans:
    for i,col in enumerate(mu.columns):
        if df_standardized[col].is_null().any():
            found_nan=True
            print(f"some value in column {col,i} is nan")

    if found_nan:
        raise RuntimeError("found nan")
    df_mean=df.with_columns([df_standardized[c] for c in df_standardized.columns])

    ScOut = f'{OutputDir}/SingleCell/'
    if not os.path.exists(ScOut): 
        os.makedirs(ScOut)
    df_mean.write_parquet(f'{ScOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}_mean.parquet')

    # # group/combine by well
    # df_agg=df.drop(columns=['Metadata_Site'])
    df_agg = df_mean
    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_PlateWell', 'Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    # # group by mean for all float features, and group by first for all non-float columns (indices and string metadata)
    group_by_aggregates=[
        *[pl.mean(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_mean=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MeanOut = f'{OutputDir}/MeanCell/'
    if not os.path.exists(MeanOut): 
        os.makedirs(MeanOut)
    df_agg_mean.write_parquet(f'{MeanOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}.parquet')
    print_time("binned mean data per well")




    # Calculating based on Medians:
    median = df_DMSO.select(float_columns).median()

    # Check for null or infinite medians and raise errors if found
    for col in median.columns:
        if median[col].is_null().any():
            raise RuntimeError(f"some median value in column {col} is nan?!")
        if median[col].is_infinite().any():
            raise RuntimeError(f"some median value in column {col} is infinite?!")

    # mad = df_DMSO.select([(pl.col(c) - pl.col(c).median()).abs().alias(c) for c in float_columns])
    mad = pl.concat([df.select((pl.col(c)-pl.col(c).median()).abs().median()) for c in df.select(float_columns).columns], how='horizontal')
    # mad = df_DMSO.select([pl.abs(pl.col(c) - median[c]).median().alias(c) for c in float_columns])


    # Ensure MAD values are not zero to avoid division by zero
    # Here we replace 0 with a small value (e.g., 0.01) instead of 1 to maintain the scale of MAD
    # mad = mad.select([pl.when(pl.col(c) == 0).then(0.01).otherwise(pl.col(c)).alias(c) for c in mad.columns])
    mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])

    # Check for null, infinite, or unexpected zero values in MAD and raise errors if found
    for i, col in enumerate(mad.columns):
        if mad[col].is_null().any():
            raise RuntimeError(f"some MAD value in column {col,i} is nan?!")
        if mad[col].is_infinite().any():
            raise RuntimeError(f"some MAD value in column {col,i} is infinite?!")
        if (mad[col] == 0).any():
            raise RuntimeError(f"unexpected 0 in column {col}")

    print_time("calculated DMSO distribution")
    df_standardized = df.with_columns([(pl.col(c) - median[c]) / (mad[c]) for c in median.columns])
    # CHecking for nans:
    for i,col in enumerate(median.columns):
        if df_standardized[col].is_null().any():
            found_nan=True
            print(f"some value in column {col,i} is nan")

    if found_nan:
        raise RuntimeError("found nan")
    df_mad=df.with_columns([df_standardized[c] for c in df_standardized.columns])    

    print_time("MAD normalized to DMSO distribution")

    # counts objects identified by segmentation, i.e. should be cells
    num_objects = df_mad.shape[0]

    fraction_objects_containing_nan=1-(num_rows_after_nan_trim/num_rows_before_nan_trim)
    print_time(f"num objects (cells) {num_objects} ({(fraction_objects_containing_nan*100):.2f}% were NaN)")

    # Save Single Cell Parquets to outputs
    ScOut = f'{OutputDir}/SingleCell/'
    if not os.path.exists(ScOut): 
        os.makedirs(ScOut)
    df_mad.write_parquet(f'{ScOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}_mad.parquet')


    df_agg = df_mad
    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    group_by_aggregates=[
        *[pl.median(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_median=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MedianOut = f'{OutputDir}/MedianCell/'
    if not os.path.exists(MedianOut): 
        os.makedirs(MedianOut)
    df_agg_median.write_parquet(f'{MedianOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}.parquet')
    print_time("binned median data per well")



lala
/share/data/cellprofiler/automation/results/PB000138/4188/5547
part1
processed metadata for 5547
/share/data/cellprofiler/automation/results/PB000142/4191/5574
part1
processed metadata for 5574
/share/data/cellprofiler/automation/results/PB000141/4187/5546
part1
processed metadata for 5546
/share/data/cellprofiler/automation/results/PB000140/4189/5573
part1
processed metadata for 5573
/share/data/cellprofiler/automation/results/PB000137/4185/5532
part1
processed metadata for 5532
/share/data/cellprofiler/automation/results/PB000139/4186/5533
part1
processed metadata for 5533
2024-08-09 15:07:37 dropped NaNs


/tmp/ipykernel_1411136/908342519.py:124: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])


2024-08-09 15:07:38 calculated DMSO distribution
2024-08-09 15:07:56 binned mean data per well


/tmp/ipykernel_1411136/908342519.py:192: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])


2024-08-09 15:08:06 calculated DMSO distribution
2024-08-09 15:08:06 MAD normalized to DMSO distribution
2024-08-09 15:08:06 num objects (cells) 207924 (0.03% were NaN)
2024-08-09 15:08:23 binned median data per well
lala
/share/data/cellprofiler/automation/results/PB000138/4188/5547
part1
processed metadata for 5547
/share/data/cellprofiler/automation/results/PB000142/4191/5574
part1
processed metadata for 5574
/share/data/cellprofiler/automation/results/PB000141/4187/5546
part1
processed metadata for 5546
/share/data/cellprofiler/automation/results/PB000140/4189/5573
part1
processed metadata for 5573
/share/data/cellprofiler/automation/results/PB000137/4185/5532
part1
processed metadata for 5532
/share/data/cellprofiler/automation/results/PB000139/4186/5533
part1
processed metadata for 5533
2024-08-09 15:10:39 dropped NaNs
2024-08-09 15:10:40 calculated DMSO distribution
2024-08-09 15:11:15 binned mean data per well
2024-08-09 15:11:30 calculated DMSO distribution
2024-08-09 15:11:31

In [39]:
# ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']

def data_processing(metaEx, cl, ObjectList, rootDir, sourceDir, OutputDir, sliceLim=7):
    #cl in metaEx.select(pl.col(['cell_line'])).unique().to_series().to_list():
    #ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
    print('lala')
    df = pl.DataFrame()
    for BaseDir in dirlist:
        mdf_op = metaEx.filter(pl.col('barcode') == BaseDir.split('/')[-1])
        image_id = mdf_op.select(pl.col('image_id')).unique().to_series().to_list()[-1]
        cp_id = mdf_op.select(pl.col('cp_id')).unique().to_series().to_list()[-1]
        print(f'{BaseDir}/{image_id}/{cp_id}')
        BaseDir = f'{BaseDir}/{image_id}/{cp_id}'


        # nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
        # cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet').add_prefix('Cytoplasm_').reset_index()
        # cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()
        # f_df=pl.read_parquet(f)
        # f_df=f_df.rename({x:f'{feature_set_name}_{x}' for x in f_df.columns})
        

        nuclei = pl.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet')
        nuclei=nuclei.rename({x:f'Nuclei_{x}' for x in nuclei.columns})
        cytoplasm = pl.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet')
        cytoplasm=cytoplasm.rename({x:f'Cytoplasm_{x}' for x in cytoplasm.columns})
        cells = pl.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet')
        cells=cells.rename({x:f'Cells_{x}' for x in cells.columns})
        # step 1: Take the mean values of 'multiple nuclei' belonging to one cell

        nuclei = nuclei.group_by([
            "Nuclei_Metadata_Barcode","Nuclei_Metadata_Well",
            "Nuclei_Parent_cells", 'Nuclei_Metadata_Site'
        ]).mean()

        df_one = cytoplasm.join(nuclei,
                    how='left', 
                    right_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Site', 'Nuclei_Parent_cells', 'Nuclei_Metadata_Barcode'],
                    left_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site', 'Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'])
                    
        df_one = df_one.join(cells, how='left', 
                        left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site','Cytoplasm_ObjectNumber', 'Cytoplasm_Metadata_Barcode'],
                        right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])

        # print_time("initial merging")
        print('part1')

        # deduplicate barcode/well/site - renamed cytoplasm_Metadata* to Metadata* and removes nuclei_* etc
        unique_metadata_feature_names=['Metadata_Barcode','Metadata_Well','Metadata_Site']
        df_one=df_one.rename({f'Cytoplasm_{suffix}':suffix for suffix in unique_metadata_feature_names})
        # df = df.filter(pl.col(''))       
        # for some reason, the site is parsed as float, even though it really should be an int
        if df_one['Metadata_Site'].dtype in [np.dtype('float32'), np.dtype('float64')]:
            # sometimes, for some reason, site indices are inf/nan
            site_is_nan_mask=np.isnan(df_one['Metadata_Site'])
            site_is_inf_mask=np.isinf(df_one['Metadata_Site'])
            
            try:
                num_sites_nan=np.sum(site_is_nan_mask)
                num_sites_inf=np.sum(site_is_inf_mask)
                assert num_sites_nan==0, f"found nan site values (n = {num_sites_nan})"
                assert num_sites_inf==0, f"found inf site values (n = {num_sites_inf})"
            except AssertionError as e:
                print(f"info - this issue was automatically circumvented in the code : {e}")
                df_one=df_one[~(site_is_inf_mask|site_is_nan_mask)]
                
            num_metadata_site_entries_nonint=np.sum(np.abs(df_one['Metadata_Site']%1.0)>1e-6)
            assert num_metadata_site_entries_nonint==0, f"ERROR : {num_metadata_site_entries_nonint} imaging sites don't have integer indices. that should not be the case, and likely indicates a bug."
            
            #Should use np.round, no? TODO ask patrick. Truncation Errors are annoying.
            df_one['Metadata_Site'] = df_one['Metadata_Site'].astype(np.dtype('int32'))
        
        #Adding Compound Metadata to each row
        df_one = df_one.join(mdf_op.rename({x:f"Metadata_cmpd_{x}" for x in mdf_op.columns}),left_on='Metadata_Well',right_on='Metadata_cmpd_well_id')
        df_one = df_one.filter(pl.col('Metadata_Site')<sliceLim)
        df = pl.concat([df, df_one])
        plate_name = f'processed metadata for {BaseDir.split("/")[-1]}'
        print(plate_name)
    ###Here should be workable to unify by cell line.
    df = df.with_columns((pl.col("Metadata_Barcode") + "_" + pl.col("Metadata_Well")).alias("Metadata_PlateWell"))
    df = df.filter(pl.col('Metadata_cmpd_cell_line')==cl)
    ###End
    # drop all rows that contain nan
    num_rows_before_nan_trim = df.shape[0]
    for col in df.select([pl.col(pl.Float32),pl.col(pl.Float64)]).columns:
        before_drop=df.shape[0]
        df=df.filter(pl.col(col).is_not_null())
        after_drop=df.shape[0]

        #num_values_dropped=before_drop-after_drop
        #if num_values_dropped>0:
        #    print(f"dropped {num_values_dropped} rows due to NaNs in column {col}")

    num_rows_after_nan_trim = df.shape[0]

    print_time("dropped NaNs")

    # Clip outliers
    if use_clipping:
        float_cols = [c for c_name,c_dtype in zip(df.columns,df.dtypes) if "float" in str(c_dtype)]
        lower_quantile = df.select(float_cols).quantile(0.01)
        upper_quantile = df.select(float_cols).quantile(0.99)
        print("calced quantiles")

        for col in float_cols:
            df = df.with_column(
                pl.col(col).clip(lower=lower_quantile[col],upper=upper_quantile[col])
            )
            
        print("clipped")
    # print(df.head())
    # for some reason, DMSO is used as batch id for cells that are not treated with a drug
    df_DMSO=df.filter(pl.col('Metadata_cmpd_cmpdname')=='dmso')
    assert df_DMSO.shape[0]>0, "did not find any wells 'treated' with DMSO"

    # Running Means here:
    mu = df_DMSO.select(float_columns).mean()
    for col in mu.columns:
        if mu[col].is_null().any():
            raise RuntimeError(f"some mean value in column {col,i} is nan?!")
        if mu[col].is_infinite().any():
            raise RuntimeError(f"some mean value in column {col,i} is infinite?!")
                
    std = df_DMSO.select(float_columns).std()
    # replace 0 with 1 (specifically not clip) to avoid div by zero
    std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])

    for i,col in enumerate(std.columns):
        if std[col].is_null().any():
            raise RuntimeError(f"some std value in column {col,i} is nan?!")
        if std[col].is_infinite().any():
            raise RuntimeError(f"some std value in column {col,i} is infinite?!")
        if (std[col]==0).any():
            raise RuntimeError(f"unexpected 0 in column {col}")

    print_time("calculated DMSO distribution")

    df_standardized = df.with_columns([(pl.col(c) - mu[c]) / (std[c]+0.01) for c in mu.columns])
    found_nan=False
    # checking nans:
    for i,col in enumerate(mu.columns):
        if df_standardized[col].is_null().any():
            found_nan=True
            print(f"some value in column {col,i} is nan")

    if found_nan:
        raise RuntimeError("found nan")
    df_mean=df.with_columns([df_standardized[c] for c in df_standardized.columns])

    ScOut = f'{OutputDir}/SingleCell/'
    if not os.path.exists(ScOut): 
        os.makedirs(ScOut)
    df_mean.write_parquet(f'{ScOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}_mean.parquet')

    # # group/combine by well
    # df_agg=df.drop(columns=['Metadata_Site'])
    df_agg = df_mean
    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_PlateWell', 'Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    # # group by mean for all float features, and group by first for all non-float columns (indices and string metadata)
    group_by_aggregates=[
        *[pl.mean(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_mean=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MeanOut = f'{OutputDir}/MeanCell/'
    if not os.path.exists(MeanOut): 
        os.makedirs(MeanOut)
    df_agg_mean.write_parquet(f'{MeanOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}.parquet')
    print_time("binned mean data per well")




    # Calculating based on Medians:
    median = df_DMSO.select(float_columns).median()

    # Check for null or infinite medians and raise errors if found
    for col in median.columns:
        if median[col].is_null().any():
            raise RuntimeError(f"some median value in column {col} is nan?!")
        if median[col].is_infinite().any():
            raise RuntimeError(f"some median value in column {col} is infinite?!")

    # mad = df_DMSO.select([(pl.col(c) - pl.col(c).median()).abs().alias(c) for c in float_columns])
    mad = pl.concat([df.select((pl.col(c)-pl.col(c).median()).abs().median()) for c in df.select(float_columns).columns], how='horizontal')
    # mad = df_DMSO.select([pl.abs(pl.col(c) - median[c]).median().alias(c) for c in float_columns])


    # Ensure MAD values are not zero to avoid division by zero
    # Here we replace 0 with a small value (e.g., 0.01) instead of 1 to maintain the scale of MAD
    # mad = mad.select([pl.when(pl.col(c) == 0).then(0.01).otherwise(pl.col(c)).alias(c) for c in mad.columns])
    mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])

    # Check for null, infinite, or unexpected zero values in MAD and raise errors if found
    for i, col in enumerate(mad.columns):
        if mad[col].is_null().any():
            raise RuntimeError(f"some MAD value in column {col,i} is nan?!")
        if mad[col].is_infinite().any():
            raise RuntimeError(f"some MAD value in column {col,i} is infinite?!")
        if (mad[col] == 0).any():
            raise RuntimeError(f"unexpected 0 in column {col}")

    print_time("calculated DMSO distribution")
    df_standardized = df.with_columns([(pl.col(c) - median[c]) / (mad[c]) for c in median.columns])
    # CHecking for nans:
    for i,col in enumerate(median.columns):
        if df_standardized[col].is_null().any():
            found_nan=True
            print(f"some value in column {col,i} is nan")

    if found_nan:
        raise RuntimeError("found nan")
    df_mad=df.with_columns([df_standardized[c] for c in df_standardized.columns])    

    print_time("MAD normalized to DMSO distribution")

    # counts objects identified by segmentation, i.e. should be cells
    num_objects = df_mad.shape[0]

    fraction_objects_containing_nan=1-(num_rows_after_nan_trim/num_rows_before_nan_trim)
    print_time(f"num objects (cells) {num_objects} ({(fraction_objects_containing_nan*100):.2f}% were NaN)")

    # Save Single Cell Parquets to outputs
    ScOut = f'{OutputDir}/SingleCell/'
    if not os.path.exists(ScOut): 
        os.makedirs(ScOut)
    df_mad.write_parquet(f'{ScOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}_mad.parquet')


    df_agg = df_mad
    df_float_columns=set(list(df_agg.select(float_columns).columns))
    group_by_columns=['Metadata_Barcode','Metadata_Well']
    other_columns=set(list(df_agg.columns))-df_float_columns-set(group_by_columns)
    group_by_aggregates=[
        *[pl.median(x) for x in list(df_float_columns)],
        *[pl.first(x) for x in list(other_columns)]
    ]
    df_agg_median=df_agg.group_by(group_by_columns).agg(group_by_aggregates)
    MedianOut = f'{OutputDir}/MedianCell/'
    if not os.path.exists(MedianOut): 
        os.makedirs(MedianOut)
    df_agg_median.write_parquet(f'{MedianOut}/{df.select(pl.col("Metadata_cmpd_cell_line")).unique().item()}.parquet')
    print_time("binned median data per well")



In [43]:
metaEx.select(pl.col(['cell_line'])).unique().to_series().to_list()


['HT29', 'HCT116']

In [44]:
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']

In [45]:
data_processing(metaEx, 'HCT116', ObjectList, rootDir, sourceDir, OutputDir)

lala
/share/data/cellprofiler/automation/results/PB000138/4188/5547
part1
processed metadata for 5547
/share/data/cellprofiler/automation/results/PB000142/4191/5574
part1
processed metadata for 5574
/share/data/cellprofiler/automation/results/PB000141/4187/5546
part1
processed metadata for 5546
/share/data/cellprofiler/automation/results/PB000140/4189/5573
part1
processed metadata for 5573
/share/data/cellprofiler/automation/results/PB000137/4185/5532
part1
processed metadata for 5532
/share/data/cellprofiler/automation/results/PB000139/4186/5533
part1
processed metadata for 5533
2024-08-09 15:47:21 dropped NaNs


/tmp/ipykernel_1411136/1475054329.py:126: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])


2024-08-09 15:47:22 calculated DMSO distribution
2024-08-09 15:47:55 binned mean data per well


/tmp/ipykernel_1411136/1475054329.py:194: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])


2024-08-09 15:48:10 calculated DMSO distribution
2024-08-09 15:48:13 MAD normalized to DMSO distribution
2024-08-09 15:48:13 num objects (cells) 370680 (0.02% were NaN)
2024-08-09 15:48:47 binned median data per well


In [46]:
data_processing(metaEx, 'HT29', ObjectList, rootDir, sourceDir, OutputDir)

lala
/share/data/cellprofiler/automation/results/PB000138/4188/5547
part1
processed metadata for 5547
/share/data/cellprofiler/automation/results/PB000142/4191/5574
part1
processed metadata for 5574
/share/data/cellprofiler/automation/results/PB000141/4187/5546
part1
processed metadata for 5546
/share/data/cellprofiler/automation/results/PB000140/4189/5573
part1
processed metadata for 5573
/share/data/cellprofiler/automation/results/PB000137/4185/5532
part1
processed metadata for 5532
/share/data/cellprofiler/automation/results/PB000139/4186/5533
part1
processed metadata for 5533
2024-08-09 15:52:57 dropped NaNs


/tmp/ipykernel_1411136/1475054329.py:126: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  std = std.select([pl.col(c).map_dict({0: 1}, default=pl.col(c)) for c in std.columns])


2024-08-09 15:52:58 calculated DMSO distribution
2024-08-09 15:53:16 binned mean data per well


/tmp/ipykernel_1411136/1475054329.py:194: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  mad = mad.select([pl.col(c).map_dict({0: 0.01}, default=pl.col(c)) for c in mad.columns])


2024-08-09 15:53:25 calculated DMSO distribution
2024-08-09 15:53:26 MAD normalized to DMSO distribution
2024-08-09 15:53:26 num objects (cells) 207924 (0.03% were NaN)
2024-08-09 15:53:46 binned median data per well


In [34]:
# df = df.filter(pl.col('Metadata_Site')<7)

In [35]:
# df.select(pl.col('Metadata_Site')).to_series().unique().to_list()

In [36]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2024-08-09 15:12:05


In [37]:
# df_meanall.head()

In [38]:
# toNpy = df_scall.loc[:,~df_scall.columns.str.contains('Metadata_')]
# toNpy.reset_index(inplace=True, drop=True)
# toNpy = toNpy.copy()
# dataNpy = toNpy.to_numpy()
# np.isnan(dataNpy).any()
# colnames = toNpy.columns